In [27]:
from QuantLib import *
import math
import calendar
import ipywidgets as widgets
from IPython.display import display
from datetime import date

# Set up market data with current date as default
current_date = date.today()
current_year = current_date.year
today_widget = widgets.Label(value=current_date.strftime("%Y-%m-%d"))

# Create widgets for gas deal specifics
    
quantity_widget = widgets.IntText(value=10000, description='Quantity (therms/day):', style={'description_width': 'initial'})
sell_price_widget = widgets.FloatText(value=102.0, description='Contract Sell Price (pence/therm):', style={'description_width': 'initial'})
spot_price_widget = widgets.FloatText(value=101.0, description='Spot Price (pence/therm):', style={'description_width': 'initial'})
volatility_widget = widgets.FloatText(value=0.25, description='Volatility:', style={'description_width': 'initial'})
days_in_contract = widgets.IntText(value=31, description='Contract Days', style={'description_width': 'initial'})
days_in_contract.disabled = True
start_month = widgets.Dropdown(
    options=[
        ('Jan', 1), ('Feb', 2), ('Mar', 3), ('Apr', 4), 
        ('May', 5), ('Jun', 6), ('Jul', 7), ('Aug', 8), 
        ('Sep', 9), ('Oct', 10), ('Nov', 11), ('Dec', 12)],
    description='Month:'
)
end_month = widgets.Dropdown(
    options=[
        ('Jan', 1), ('Feb', 2), ('Mar', 3), ('Apr', 4), 
        ('May', 5), ('Jun', 6), ('Jul', 7), ('Aug', 8), 
        ('Sep', 9), ('Oct', 10), ('Nov', 11), ('Dec', 12)],
    value = 2,
    description='Month:'
)
start_year = widgets.Dropdown(
    options=[str(current_year), str(current_year + 1), str(current_year + 2)],
    value=str(current_year),
    description='Start Year:',
    disabled=False,
)
end_year = widgets.Dropdown(
    options=[str(current_year), str(current_year + 1), str(current_year + 2)],
    value=str(current_year),
    description='End Year:',
    disabled=False,
)
start_date = widgets.Label(value=date(int(start_year.value), int(start_month.value), 1).strftime("%Y-%m-%d"))
start_contract = widgets.HBox([start_month, start_year, start_date])
end_date = widgets.Label(value=date(int(end_year.value), int(end_month.value), 1).strftime("%Y-%m-%d"))
end_contract = widgets.HBox([end_month, end_year, end_date])
warning = widgets.Label()


def last_day_of_month(year, month):
    _, last_day = calendar.monthrange(year, month)
    return date(year, month, last_day)

def check_contract_date():
    sdate = date(int(start_year.value), int(start_month.value), 1)
    edate = last_day_of_month(int(end_year.value), int(end_month.value))
    cdate = date(current_date.year, current_date.month, 1)
    error = "Contract Date is ok"
    if sdate > edate:
        error = "The start period can't be after the ending period"
    elif sdate <= cdate:
        error = "The start period must start in the next month when compared with the current date" 
    warning.value = error
    days_in_contract.value = str((edate - sdate).days + 1)

check_contract_date()

def on_change_start():
    start_date.value = date(int(start_year.value), int(start_month.value), 1).strftime("%Y-%m-%d")
    check_contract_date()

def on_change_end():
    end_date.value = last_day_of_month(int(end_year.value), int(end_month.value)).strftime("%Y-%m-%d")
    check_contract_date()

start_month.on_trait_change(on_change_start)
start_year.on_trait_change(on_change_start)
end_month.on_trait_change(on_change_end)
end_year.on_trait_change(on_change_end)

# Create accordions
settings_accordion = widgets.Accordion(children=[today_widget])
settings_accordion.set_title(0, 'Settings')

gas_deal_accordion = widgets.Accordion(children=[widgets.VBox(
    [quantity_widget, 
     sell_price_widget, 
     spot_price_widget, 
     volatility_widget,
     days_in_contract,
     start_contract,
     end_contract,
     warning])])
gas_deal_accordion.set_title(0, 'Gas Deal')

# Display accordions
display(settings_accordion, gas_deal_accordion)

# Function to update variables based on widget values
def update_variables():
    today = Date(today_widget.value.day, today_widget.value.month, today_widget.value.year)
    Settings.instance().evaluationDate = today
    quantity = quantity_widget.value
    sell_price = sell_price_widget.value
    spot_price = spot_price_widget.value
    volatility = volatility_widget.value
    days_in_march = days_in_march_widget.value
    return today, quantity, sell_price, spot_price, volatility, days_in_march

Accordion(children=(Label(value='2025-01-27'),), titles=('Settings',))

Accordion(children=(VBox(children=(IntText(value=10000, description='Quantity (therms/day):', style=Descriptio…

In [4]:
total_quantity = quantity * days_in_march

# Create a flat forward curve for simplicity
day_counter = Actual365Fixed()
calendar = UnitedKingdom()
# spot_curve = FlatForward(today, spot_price, day_counter, Continuous, Annual)
# ts_curve = YieldTermStructureHandle(spot_curve)

# Option parameters for daily options
strike = spot_price  # ATM option for simplicity

# Black-Scholes process for option pricing
u = SimpleQuote(spot_price)
r = SimpleQuote(0.0)  # Assuming no interest rate for simplicity
sigma = SimpleQuote(volatility)
riskFreeCurve = FlatForward(today, r.value(), day_counter)
dividendYield = FlatForward(today, 0.0, day_counter)

# Use BlackConstantVol instead of FlatVol
black_vol = BlackConstantVol(today, calendar, sigma.value(), day_counter)

stochasticProcess = BlackScholesMertonProcess(QuoteHandle(u), 
                                              YieldTermStructureHandle(dividendYield), 
                                              YieldTermStructureHandle(riskFreeCurve), 
                                              BlackVolTermStructureHandle(black_vol))

# Function to price daily options
def price_daily_option(date):
    expiry_date = date
    option = EuropeanOption(PlainVanillaPayoff(Option.Call, strike), 
                            EuropeanExercise(expiry_date))
    engine = AnalyticEuropeanEngine(stochasticProcess)
    option.setPricingEngine(engine)
    print(date, option.NPV())
    return option.NPV()

# Calculate option prices for each day in March
total_option_premium = 0
for i in range(1, days_in_march + 1):
    expiry_date = Date(i, 3, 2025)
    daily_option_price = price_daily_option(expiry_date)
    total_option_premium += quantity * daily_option_price

# Profit calculation for the deal
profit = (sell_price - spot_price) * total_quantity - total_option_premium

print(f"Total Option Premium for March: £ {total_option_premium/100:.2f}")
print(f"Estimated Profit/Loss: £ {profit/100:.2f}")

NameError: name 'quantity' is not defined

In [8]:
from tradingview_ta import TA_Handler, Interval, Exchange
import datetime

def get_gwm_futures_quotes():
    futures_quotes = {}
    current_date = datetime.date(2025, 1, 24)  # Current date from the given context
    
    # Correct month codes for futures
    month_codes = {
        1: 'F', 2: 'G', 3: 'H', 4: 'J', 5: 'K', 6: 'M',
        7: 'N', 8: 'Q', 9: 'U', 10: 'V', 11: 'X', 12: 'Z'
    }
    
    for month in range(1, 13):  # Loop through 12 months
        future_date = current_date.replace(month=month)
        if future_date < current_date:
            future_date = future_date.replace(year=future_date.year + 1)
        
        month_code = month_codes[future_date.month]
        symbol = f"GWM{month_code}{str(future_date.year)}"
        
        handler = TA_Handler(
            symbol=symbol,
            exchange="ICEEUR",
            screener="futures",
            interval=Interval.INTERVAL_1_DAY
        )
        
        try:
            analysis = handler.get_analysis()
            if analysis.indicators:
                futures_quotes[symbol] = {
                    "close": analysis.indicators["close"],
                    "open": analysis.indicators["open"],
                    "high": analysis.indicators["high"],
                    "low": analysis.indicators["low"],
                    "volume": analysis.indicators["volume"]
                }
        except Exception as e:
            print(f"Error fetching data for {symbol}: {str(e)}")
    
    return futures_quotes

# Fetch all GWM futures quotes
gwm_quotes = get_gwm_futures_quotes()

# Print the results
for symbol, data in gwm_quotes.items():
    print(f"{symbol}:")
    for key, value in data.items():
        print(f"  {key}: {value}")
    print()

Error fetching data for GWMF2025: Exchange or symbol not found.
GWMG2025:
  close: 124.73
  open: 124.6
  high: 125.74
  low: 121.29
  volume: 15725

GWMH2025:
  close: 123.12
  open: 123.65
  high: 124.4
  low: 119.79
  volume: 11150

GWMJ2025:
  close: 122.44
  open: 123.03
  high: 123.6
  low: 119.36
  volume: 5110

GWMK2025:
  close: 122.11
  open: 123.1
  high: 123.1
  low: 119.18
  volume: 5380

GWMM2025:
  close: 121.78
  open: 118.83
  high: 122.31
  low: 118.81
  volume: 2435

GWMN2025:
  close: 120.81
  open: 121.59
  high: 121.59
  low: 120.81
  volume: 585

GWMQ2025:
  close: 121.09
  open: 121.09
  high: 121.09
  low: 121.09
  volume: 575

GWMU2025:
  close: 122.39
  open: 122.39
  high: 122.39
  low: 122.39
  volume: 575

GWMV2025:
  close: 117.39
  open: 117.39
  high: 117.39
  low: 117.39
  volume: 375

GWMX2025:
  close: 116.76
  open: 116.76
  high: 116.76
  low: 116.76
  volume: 375

GWMZ2025:
  close: 116.76
  open: 116.76
  high: 116.76
  low: 116.76
  volume: 285
